In [ ]:
!pip install transformers==4.2.2
# !pip install git+https://github.com/huggingface/transformers

     |████████████████████████████████| 1.8 MB 6.9 MB/s 
     |████████████████████████████████| 2.9 MB 70.5 MB/s 
     |████████████████████████████████| 895 kB 70.5 MB/s 


In [ ]:
!nvidia-smi

Tue Jan 25 09:17:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import re
import os
import json
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelForCausalLM
from transformers import pipeline

In [ ]:
IN_COLAB: bool = False
try:
  from google.colab import drive, files
  IN_COLAB = True
except:
  IN_COLAB = False

path: str = ''
if IN_COLAB:
  drive.mount('/content/drive')
  path = 'drive/MyDrive/RapMachine/keywords.json'
else:
  path = None
df: pd.DataFrame = pd.read_json(path)
df = df.rename(columns={'Text': 'text', 'Keywords': 'keywords'})
to_drop: list = df[list(map(lambda e: type(e) != list, df['keywords'].tolist()))].index
df = df.drop(to_drop)

Mounted at /content/drive


# Working

In [ ]:
df

,text,keywords
0,"(Lil momma\nLil momma, hey\nFood, yeah)\nWhoa,...","[collard, pea, momma, sauce, wing]"
1,"You know what I'm sayin', most of you niggas i...","[niggas, cappin, rappin, truth, kitch]"
2,"I'ma pull up in a Benz, ayy, ayy Unlimited , s...","[bounce, friend, shoe, thriller, payin]"
3,Nigga\nNigga I feel like mothafuckin' Dapper D...,"[pay, bitch, brick, money, treat]"
4,Yeah\nUh whassup Tryna pull up in the Tesla wi...,"[reup, plug, people, foot, drummer]"
...,...,...
71117,"'s like a typo\nFlow is universal, take a bloo...","[service, teach, outta, deaf, leopard]"
71118,"So tell me Slim, whatchu wanna do\nwhen you kn...","[honey, players, sugar, errtime, drink]"
71119,"""WE HAAAAAAAVE A WHOLE LOTTA superstars, on th...","[stage, tonight, flirt, fllow, flizzow]"
71120,"I wanna see you again, see you again Dont you ...","[wonder, lady, heaven, journey, connection]"


In [ ]:


directory: str = 'training_data/'
if not os.path.exists(directory):
  os.makedirs(directory)

def build_text_file(df: pd.DataFrame, dest_path: str) -> None:
  text: str = ''
  for _, row in df.iterrows():
    keywords: str = row[1]
    lyrics: str = row[0]
    line: str = str(lyrics.strip())
    line = re.sub(r"\s", " ", line)
    text += line + "\n"
    # processed: str  = re.sub(r"\n", " ", lyrics.strip())
    # processed: str = re.sub(r'\n', '<|newline|>', lyrics)
    # prompt: str = f"<|startoftext|>CONTEXT: {keywords} RAP-LYRICS: {processed}<|endoftext|>\n"
    # text += prompt
    # text += processed
  f = open(dest_path, 'w')
  f.write(text)
  f.close()


train, test = train_test_split(df,test_size=0.15) 
build_text_file(train, './training_data/data_train.txt')
build_text_file(test, './training_data/data_test.txt')
pass

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'gpt2')
    # bos_token='<|startoftext|>',
    # eos_token='<|endoftext|>',)
    # additional_special_tokens=['<|newline|>'])

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
del df

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=4)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=4)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_path = './training_data/data_train.txt'
test_path = './training_data/data_test.txt'
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (38063719 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
model_path = 'drive/MyDrive/RapMachine/RM'

In [ ]:
model = AutoModelForCausalLM.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))


training_args = TrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=1, # number of training epochs
    per_device_train_batch_size=64, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 5000, # Number of update steps between two evaluations.
    save_steps=5000, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    save_total_limit=3
    )

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()
trainer.save_model()

Step,Training Loss
500,5.911100
1000,5.617900
1500,5.511100
2000,5.459600
2500,5.412200
3000,5.375500
3500,5.321800
4000,5.327400
4500,5.322900
5000,5.295300


Step,Training Loss
500,5.911100
1000,5.617900
1500,5.511100
2000,5.459600
2500,5.412200
3000,5.375500
3500,5.321800
4000,5.327400
4500,5.322900
5000,5.295300


In [ ]:
rapper = pipeline('text-generation',model=model_path, tokenizer='gpt2',config={"num_beams":5, 'min_length': 200})

In [ ]:
input: str = '<|startoftext|>CONTEXT: fore it crash, sky falling, Drop that ass, fore, higher, falling, Drop, crash, sky, ass, baby, switched my robot RAP-LYRICS:'
r = rapper('Straight')
output: str = str(r[0]['generated_text'])
print(output)
print(len(output))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Straight, and now's a nigga can't find a way But even wit me now, don't touch you, I got the gwaan a hand to hang and feet all that'll never make me see me a chick, she can
172


# Test Prompt

In [ ]:
rapper = pipeline(
    'text-generation',
    model=model_path,
    tokenizer='gpt2',
    config={
        "num_beams": 5,
        'min_length': 400,
        'top-k': 50})

In [ ]:
keywords: str = 'fore it crash, sky falling, Drop that ass, fore, higher, falling, Drop, crash, sky, ass, baby'
input: str = f"CONTEXT: {keywords}\nRAP-LYRICS:"
r = rapper(input)
output: str = str(r[0]['generated_text'])
output

# Data Doc

- Config 1
5 Epochs and substituting '\n' characters to spaces, while keeping the prompt format `CONTEXT: {keywords}\nRAP-LYRICS: {lyrics}` leads to short output. However, the quality of the output is okish.

- Config 2
bos and eos are added to the tokenizer. Input is also modified to `<|startoftext|>CONTEXT: {keywords} RAP-LYRICS: {lyrics}<|endoftext|>\n`. Training with 5 Epochs.
  - `<|newline|>` character apperars. Text is still too short.
- Config 3
Config 2 tested again, reloaded, and with 3 epochs only. Still too short

- Config 4
 `<|startoftext|>CONTEXT: {keywords} RAP-LYRICS: {lyrics}<|endoftext|>\n`. lyrics
 will be preprocessesed with `<|newline|>` character. Epchs: 3. Still too short.

- Config 5
 `<|startoftext|>CONTEXT: {keywords} RAP-LYRICS: {lyrics}<|endoftext|>\n`. lyrics
 will be preprocessesed with `<|newline|>` character but not included in tokenizer. Epchs: 3. Still too short.
 - Config 6
 Feeding ONLY plain text without newlines and no prompts. 2 Epochs, GPT2, Steps reduced to 200. min_length=200 and beam search. Works properly